In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
from operationUnits import * 

nodes, edges = parse_ctx()

In [4]:
nodes

[{'id': 1,
  'label': 'IAB-HI',
  'properties': {'kind': 'START',
   'in': [],
   'out': ['IAB-HI'],
   'temperature': 300,
   'pressure': '3445.05',
   'composition': [[1, 0.9], [2, 0.1]]}},
 {'id': 2,
  'label': 'IC-HI',
  'properties': {'kind': 'START',
   'in': [],
   'out': ['IC-HI'],
   'temperature': 300,
   'pressure': '3445.05',
   'composition': [[3, 1.0]]}},
 {'id': 3,
  'label': 'HX1-IC',
  'properties': {'kind': 'HX',
   'in': ['IC-HI'],
   'out': ['IC'],
   'description': 'Heater for Toluene feed stream'}},
 {'id': 4,
  'label': 'HX0-IAB',
  'properties': {'kind': 'HX',
   'in': ['IAB-HI'],
   'out': ['IAB'],
   'description': 'Heater for Hydrogen feed stream'}},
 {'id': 5,
  'label': 'R1',
  'properties': {'kind': 'CONREACTOR',
   'in': ['FRT'],
   'out': ['INTER1'],
   'description': 'Reactor 1'}},
 {'id': 6,
  'label': 'R2',
  'properties': {'kind': 'CONREACTOR',
   'in': ['INTER1'],
   'out': ['ABCDE'],
   'description': 'Reactor 2'}},
 {'id': 7,
  'label': 'HX2-ABCDE

In [8]:
edges

[{'id': 1, 'from': 1, 'to': 4},
 {'id': 2, 'from': 2, 'to': 3},
 {'id': 3, 'from': 3, 'to': 18},
 {'id': 4, 'from': 4, 'to': 18},
 {'id': 5, 'from': 5, 'to': 6},
 {'id': 6, 'from': 6, 'to': 8},
 {'id': 7, 'from': 7, 'to': 8},
 {'id': 8, 'from': 8, 'to': 9},
 {'id': 9, 'from': 8, 'to': 12},
 {'id': 10, 'from': 9, 'to': 10},
 {'id': 11, 'from': 10, 'to': 11},
 {'id': 12, 'from': 11, 'to': 18},
 {'id': 13, 'from': 12, 'to': 13},
 {'id': 14, 'from': 13, 'to': 14},
 {'id': 15, 'from': 13, 'to': 19},
 {'id': 16, 'from': 14, 'to': 20},
 {'id': 17, 'from': 14, 'to': 15},
 {'id': 18, 'from': 14, 'to': 16},
 {'id': 19, 'from': 15, 'to': 16},
 {'id': 20, 'from': 16, 'to': 16},
 {'id': 21, 'from': 16, 'to': 15},
 {'id': 22, 'from': 16, 'to': 16},
 {'id': 23, 'from': 16, 'to': 16},
 {'id': 24, 'from': 17, 'to': 16},
 {'id': 25, 'from': 17, 'to': 18},
 {'id': 26, 'from': 18, 'to': 5}]

In [12]:
text = """
  CONREACTOR UID=R2, NAME=Reactor 2
      FEED INTER1
      PRODUCT  M=ABCDE-HI
      OPERATION ISOTHERMAL
      RXCALCULATION MODEL=STOIC
      RXSTOIC RXSET=2
      REACTION 2
      BASE COMPONENT=4
      CONVERSION 0.15
"""
parse_conreactor_block(text)

{'id': None,
 'label': 'R2',
 'properties': {'kind': 'CONREACTOR',
  'in': ['INTER1'],
  'out': ['ABCDE-HI'],
  'description': 'Reactor 2'}}

In [14]:
import re
from pprint import pprint
from operationUnits import * 

# Read text content from the file
file_path = "sample_full.txt"
with open(file_path, "r") as file:
    text_content = file.read()

parse_pro2inp(text_content)

# a = slice_section(text_content)

NameError: name 'parse_pro2inp' is not defined

In [ ]:
test(text_content)

STREAM DATA
  PROPERTY STREAM=IAB-HI, TEMPERATURE=300, PRESSURE=3445.05, PHASE=M, &
         RATE(M)=100, COMPOSITION(M)=1,0.9/2,0.1
  PROPERTY STREAM=IC-HI, TEMPERATURE=300, PRESSURE=3445.05, PHASE=M, &
         RATE(M)=100, COMPOSITION(M)=3,1


In [ ]:
sections = slice_section(text_content)


for text in sections:
    if text.startswith("STREAM DATA"):
        print(text)
        result = parse_stream_data(text)
        print(result)
        t = text



STREAM DATA
  PROPERTY STREAM=IAB-HI, TEMPERATURE=300, PRESSURE=3445.05, PHASE=M, &
         RATE(M)=100, COMPOSITION(M)=1,0.9/2,0.1
  PROPERTY STREAM=IC-HI, TEMPERATURE=300, PRESSURE=3445.05, PHASE=M, &
         RATE(M)=100, COMPOSITION(M)=3,1
[]


'STREAM DATA\n  PROPERTY STREAM=IAB-HI, TEMPERATURE=300, PRESSURE=3445.05, PHASE=M, &\n         RATE(M)=100, COMPOSITION(M)=1,0.9/2,0.1\n  PROPERTY STREAM=IC-HI, TEMPERATURE=300, PRESSURE=3445.05, PHASE=M, &\n         RATE(M)=100, COMPOSITION(M)=3,1'

In [ ]:
parse_stream_data(t)

[{'label': 'IAB-HI',
  'temperature': 300,
  'pressure': '3445.05',
  'composition': [[1, 0.9], [2, 0.1]]},
 {'label': 'IC-HI',
  'temperature': 300,
  'pressure': '3445.05',
  'composition': [[3, 1.0]]}]

In [ ]:
def parse_stream_data(text_content):
    # Define a list to store the parsed stream data
    streams = []

    # Regular expression to match each PROPERTY STREAM block
    pattern = re.compile(
        r"STREAM=(?P<label>[^,]+),\s*"
        r"TEMPERATURE=(?P<temperature>[\d.]+),\s*"
        r"PRESSURE=(?P<pressure>[\d.]+),\s*"
        r"PHASE=[^,]+,\s*&\s*"
        r"RATE\(M\)=[\d.]+,\s*"
        r"COMPOSITION\(M\)=(?P<composition>.+)"
    )

    # Iterate over all matches
    for match in pattern.finditer(text_content):
        # Extract composition and parse into nested lists
        raw_composition = match.group("composition")
        composition = [
            [int(pair.split(",")[0]), float(pair.split(",")[1])]
            for pair in raw_composition.split("/")
        ]

        # Add the parsed data to the streams list
        streams.append({
            "label": match.group("label"),
            "temperature": int(match.group("temperature")),
            "pressure": match.group("pressure"),
            "composition": composition
        })

    return streams


In [ ]:
unit_operations = [
    {'unit': 'HX', 'uid': 'HX1-IC', 'name': 'Heater for Toluene feed stream', 'inlets': ['IC-HI'], 'outlets': ['IC']},
    {'unit': 'HX', 'uid': 'HX0-IAB', 'name': 'Heater for Hydrogen feed stream', 'inlets': ['IAB-HI'], 'outlets': ['IAB']},
    {'unit': 'MIXER', 'uid': 'M1-R1', 'name': 'Mixer before reactor', 'inlets': ['IC', 'IAB'], 'outlet': ['FRT']}
]

start_nodes = ['IC-HI', 'IAB-HI']

# Initialize empty lists for nodes and edges
nodes = []
edges = []

# Helper function to create nodes
def create_node(node_id, label, kind):
    return {
        "id": node_id,
        "label": label,
        "properties": {
            "kind": kind
        }
    }

# Add start nodes
node_id = 1
for start in start_nodes:
    nodes.append(create_node(node_id, start, "start"))
    node_id += 1

# Add unit operation nodes
for operation in unit_operations:
    if operation['unit'] == 'HX':
        kind = 'hx'
    elif operation['unit'] == 'MIXER':
        kind = 'mixer'
    nodes.append(create_node(node_id, operation['uid'], kind))
    node_id += 1

# Helper function to create edges
def create_edge(edge_id, from_node, to_node, label):
    return {
        "id": edge_id,
        "from": from_node,
        "to": to_node,
        "label": label
    }

# Add edges based on inlets and outlets
edge_id = 1
for operation in unit_operations:
    # Add edges from inlets to unit operations
    for inlet in operation['inlets']:
        from_node = start_nodes.index(inlet) + 1 if inlet in start_nodes else next((n['id'] for n in nodes if n['label'] == inlet), None)
        edges.append(create_edge(edge_id, from_node, node_id - len(unit_operations) + operation['uid'], inlet))
        edge_id += 1
    
    # Add edges from unit operations to outlets
    for outlet in operation['outlets']:
        from_node = node_id - len(unit_operations) + operation['uid']
        to_node = next((n['id'] for n in nodes if n['label'] == outlet), None)
        edges.append(create_edge(edge_id, from_node, to_node, outlet))
        edge_id += 1

# Construct the final graph structure
graph = {
    "nodes": nodes,
    "edges": edges
}

# Display the result
import json
print(json.dumps(graph, indent=2))


TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [ ]:
unit_operations = [
    {'unit': 'HX', 'uid': 'HX1-IC', 'name': 'Heater for Toluene feed stream', 'inlets': ['IC-HI'], 'outlets': ['IC']},
    {'unit': 'HX', 'uid': 'HX0-IAB', 'name': 'Heater for Hydrogen feed stream', 'inlets': ['IAB-HI'], 'outlets': ['IAB']},
    {'unit': 'MIXER', 'uid': 'M1-R1', 'name': 'Mixer before reactor', 'inlets': ['IC', 'IAB'], 'outlets': ['FRT']}
]

start_nodes = ['IC-HI', 'IAB-HI']

# Initialize empty lists for nodes and edges
nodes = []
edges = []

# Helper function to create nodes
def create_node(node_id, label, kind):
    return {
        "id": node_id,
        "label": label,
        "properties": {
            "kind": kind
        }
    }

# Add start nodes
node_id = 1
for start in start_nodes:
    nodes.append(create_node(node_id, start, "start"))
    node_id += 1

# Add unit operation nodes
for operation in unit_operations:
    if operation['unit'] == 'HX':
        kind = 'hx'
    elif operation['unit'] == 'MIXER':
        kind = 'mixer'
    nodes.append(create_node(node_id, operation['uid'], kind))
    node_id += 1

nodes

[{'id': 1, 'label': 'IC-HI', 'properties': {'kind': 'start'}},
 {'id': 2, 'label': 'IAB-HI', 'properties': {'kind': 'start'}},
 {'id': 3, 'label': 'HX1-IC', 'properties': {'kind': 'hx'}},
 {'id': 4, 'label': 'HX0-IAB', 'properties': {'kind': 'hx'}},
 {'id': 5, 'label': 'M1-R1', 'properties': {'kind': 'mixer'}}]

In [ ]:
operation

{'unit': 'MIXER',
 'uid': 'M1-R1',
 'name': 'Mixer before reactor',
 'inlets': ['IC', 'IAB'],
 'outlets': ['FRT']}

In [ ]:

operation['inlets']

['IC', 'IAB']

In [ ]:
operation['outlets']

['FRT']

In [ ]:
next((n['id'] for n in nodes if n['label'] == inlet), None)

In [ ]:
unit_operations

[{'unit': 'HX',
  'uid': 'HX1-IC',
  'name': 'Heater for Toluene feed stream',
  'inlets': ['IC-HI'],
  'outlets': ['IC']},
 {'unit': 'HX',
  'uid': 'HX0-IAB',
  'name': 'Heater for Hydrogen feed stream',
  'inlets': ['IAB-HI'],
  'outlets': ['IAB']},
 {'unit': 'MIXER',
  'uid': 'M1-R1',
  'name': 'Mixer before reactor',
  'inlets': ['IC', 'IAB'],
  'outlets': ['FRT']}]

In [ ]:
[
  {'unit': 'START',
  'uid': 'IC-HI',
  'name': 'Start point',
  'inlets': [],
  'outlets': ['IC-HI']},
  {'unit': 'START',
  'uid': 'IAB-HI',
  'name': 'Start point',
  'inlets': [],
  'outlets': ['IAB-HI']},
  {'unit': 'HX',
  'uid': 'HX1-IC',
  'name': 'Heater for Toluene feed stream',
  'inlets': ['IC-HI'],
  'outlets': ['IC']},
 {'unit': 'HX',
  'uid': 'HX0-IAB',
  'name': 'Heater for Hydrogen feed stream',
  'inlets': ['IAB-HI'],
  'outlets': ['IAB']},
 {'unit': 'MIXER',
  'uid': 'M1-R1',
  'name': 'Mixer before reactor',
  'inlets': ['IC', 'IAB'],
  'outlets': ['FRT']}
  {'unit': 'END',
  'uid': 'FRT',
  'name': 'End point',
  'inlets': ['FRT'],
  'outlets': []}
]

In [ ]:
nodes

[{'id': 1, 'label': 'IC-HI', 'properties': {'kind': 'start'}},
 {'id': 2, 'label': 'IAB-HI', 'properties': {'kind': 'start'}},
 {'id': 3, 'label': 'HX1-IC', 'properties': {'kind': 'hx'}},
 {'id': 4, 'label': 'HX0-IAB', 'properties': {'kind': 'hx'}},
 {'id': 5, 'label': 'M1-R1', 'properties': {'kind': 'mixer'}}]

In [ ]:
inlet

'IAB'

In [ ]:
nodes

[{'id': 1, 'label': 'IC-HI', 'properties': {'kind': 'start'}},
 {'id': 2, 'label': 'IAB-HI', 'properties': {'kind': 'start'}},
 {'id': 3, 'label': 'HX1-IC', 'properties': {'kind': 'hx'}},
 {'id': 4, 'label': 'HX0-IAB', 'properties': {'kind': 'hx'}},
 {'id': 5, 'label': 'M1-R1', 'properties': {'kind': 'mixer'}}]

In [ ]:
next((n['id'] for n in nodes if n['label'] == inlet), None)

In [ ]:

# Helper function to create edges
def create_edge(edge_id, from_node, to_node, label):
    return {
        "id": edge_id,
        "from": from_node,
        "to": to_node,
        "label": label
    }

# Add edges based on inlets and outlets
edge_id = 1
for operation in unit_operations:
    # Add edges from inlets to unit operations
    for inlet in operation['inlets']:
        # Find the correct node ID for the inlet
        from_node = next((n['id'] for n in nodes if n['label'] == inlet), None)
        # Find the node ID for the unit operation
        to_node = next((n['id'] for n in nodes if n['label'] == operation['uid']), None)
        edges.append(create_edge(edge_id, from_node, to_node, inlet))
        edge_id += 1
    
    # Add edges from unit operations to outlets
    for outlet in operation['outlets']:
        from_node = next((n['id'] for n in nodes if n['label'] == operation['uid']), None)
        to_node = next((n['id'] for n in nodes if n['label'] == outlet), None)
        edges.append(create_edge(edge_id, from_node, to_node, outlet))
        edge_id += 1

# Construct the final graph structure
graph = {
    "nodes": nodes,
    "edges": edges
}

# Display the result
import json
print(json.dumps(graph, indent=2))

{
  "nodes": [
    {
      "id": 1,
      "label": "IC-HI",
      "properties": {
        "kind": "start"
      }
    },
    {
      "id": 2,
      "label": "IAB-HI",
      "properties": {
        "kind": "start"
      }
    },
    {
      "id": 3,
      "label": "HX1-IC",
      "properties": {
        "kind": "hx"
      }
    },
    {
      "id": 4,
      "label": "HX0-IAB",
      "properties": {
        "kind": "hx"
      }
    },
    {
      "id": 5,
      "label": "M1-R1",
      "properties": {
        "kind": "mixer"
      }
    }
  ],
  "edges": [
    {
      "id": 1,
      "from": 1,
      "to": 3,
      "label": "IC-HI"
    },
    {
      "id": 2,
      "from": 3,
      "to": null,
      "label": "IC"
    },
    {
      "id": 3,
      "from": 2,
      "to": 4,
      "label": "IAB-HI"
    },
    {
      "id": 4,
      "from": 4,
      "to": null,
      "label": "IAB"
    },
    {
      "id": 5,
      "from": null,
      "to": 5,
      "label": "IC"
    },
    {
      "id": 6,
  